# Research Assistant

## Review

We've covered a few major LangGraph themes:

* Memory
* Human-in-the-loop
* Controllability

Now, we'll bring these ideas together to tackle one of AI's most popular applications: research automation. 

Research is often laborious work offloaded to analysts. AI has considerable potential to assist with this.

However, research demands customization: raw LLM outputs are often poorly suited for real-world decision-making workflows. 

Customized, AI-based [research and report generation](https://jxnl.co/writing/2024/06/05/predictions-for-the-future-of-rag/#reports-over-rag) workflows are a promising way to address this.

## Goal

Our goal is to build a lightweight, multi-agent system around chat models that customizes the research process.

`Source Selection` 
* Users can choose any set of input sources for their research.
  
`Planning` 
* Users provide a topic, and the system generates a team of AI analysts, each focusing on one sub-topic.
* `Human-in-the-loop` will be used to refine these sub-topics before research begins.
  
`LLM Utilization`
* Each analyst will conduct in-depth interviews with an expert AI using the selected sources.
* The interview will be a multi-turn conversation to extract detailed insights as shown in the [STORM](https://arxiv.org/abs/2402.14207) paper.
* These interviews will be captured in a using `sub-graphs` with their internal state. 
   
`Research Process`
* Experts will gather information to answer analyst questions in `parallel`.
* And all interviews will be conducted simultaneously through `map-reduce`.

`Output Format` 
* The gathered insights from each interview will be synthesized into a final report.
* We'll use customizable prompts for the report, allowing for a flexible output format. 

![Screenshot 2024-08-26 at 7.26.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb164d61c93d48e604091_research-assistant1.png)

In [5]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python wikipedia langchain_tavily langchain_groq

## **Setup**

In [45]:
import os, operator, warnings, getpass
from typing import List, TypedDict, Annotated
from langgraph.graph import MessagesState
from pydantic import BaseModel, Field
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langchain_tavily import TavilySearch
from langchain_community.document_loaders import WikipediaLoader
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Send
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from langchain_core.messages import get_buffer_string
from IPython.display import Image, display, Markdown
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from prompt import *

warnings.filterwarnings("ignore")

In [2]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGSMITH_API_KEY")

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm.invoke("Hello, how are you?").content

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

We'll use [LangSmith](https://docs.langchain.com/langsmith/home) for [tracing](https://docs.langchain.com/langsmith/observability-concepts).

In [4]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "research_assistant"

## **Generate Analysts: Human-In-The-Loop**

Create analysts and review them using human-in-the-loop.

In [5]:
class Analysts(BaseModel):
    affiliation: str = Field(..., description="Primary affiliation of the analyst.")
    name: str = Field(..., description="Name of the analyst.")
    role: str = Field(..., description="Role of the analyst in the context of topic.")
    description: str = Field(..., description="Description of the analyst focus, concerns, and motives.")

    @property
    def personality(self):
        return f"Name: {self.name}, \nRole: {self.role}, \nDescription: {self.description} \nAffiliation: {self.affiliation}"

class Perspectives(BaseModel):
    analyst: List[Analysts] = Field(..., description="comprehensive list of analysts with their roles, descriptions, and affiliations.")

In [6]:
class GenerateAnalystState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[Analysts]

In [12]:
def create_analyst(state: GenerateAnalystState) -> dict:
    """Create an analyst to generate a report."""
    topic = state["topic"]
    max_analysts = state["max_analysts"]
    human_analyst_feedback = state.get("human_analyst_feedback", "")

    # Create a structured LLM
    structured_llm = llm.with_structured_output(Perspectives)
    
    # system prompt
    system_prompt = SystemMessage(
        content=ANALYST_INSTRUCTIONS.format(
            topic=topic,
            max_analysts=max_analysts,
            human_analyst_feedback=human_analyst_feedback,
        ))
    
    # human message
    human_message = HumanMessage(content="Create a set of analysts.")

    # create the analyst
    analysts = structured_llm.invoke([system_prompt, human_message])

    return {"analysts": analysts.analyst }

In [13]:
analysts = create_analyst({
    "topic": "AI Agents",
    "max_analysts": 2,
})

In [17]:
for analyst in analysts.get("analysts"):
    print(f"Name: {analyst.name}")
    print(f"Affiliation: {analyst.affiliation}")
    print(f"Role: {analyst.role}")
    print(f"Description: {analyst.description}")
    print("-" * 100)

Name: Dr. Emily Carter
Affiliation: Tech Innovations Lab
Role: AI Ethics Researcher
Description: Dr. Carter focuses on the ethical implications of AI agents, exploring how they impact society, privacy, and human rights. She is concerned with ensuring that AI technologies are developed and deployed responsibly, advocating for transparency and accountability in AI systems.
----------------------------------------------------------------------------------------------------
Name: Mr. James Liu
Affiliation: Future of Work Institute
Role: AI Workforce Analyst
Description: Mr. Liu analyzes the impact of AI agents on the workforce, studying how automation and AI technologies are reshaping job markets and employment trends. He aims to provide insights on how businesses and workers can adapt to the changing landscape brought about by AI advancements.
----------------------------------------------------------------------------------------------------


In [18]:
def human_feedback(state: GenerateAnalystState):
    """ No-op node that should be interrupted on """
    pass

def should_continue(state: GenerateAnalystState):
    """ Return the next node to execute """
    
    # Check if human feedback is present
    if state.get("human_analyst_feedback", None):
        return "create_analyst"
    return END

In [29]:
# Add nodes and edges 
analyst_builder = StateGraph(GenerateAnalystState)

# Add nodes
analyst_builder.add_node("create_analyst", create_analyst)
analyst_builder.add_node("human_feedback", human_feedback)

# Add edges
analyst_builder.add_edge(START, "create_analyst")
analyst_builder.add_edge("create_analyst", "human_feedback")
analyst_builder.add_conditional_edges("human_feedback", should_continue, ["create_analyst", END])

# Compile graph
memory = MemorySaver()
analyst_graph = analyst_builder.compile(checkpointer=memory, interrupt_before=["human_feedback"])

In [30]:
# View
# display(Image(analyst_graph.get_graph(xray=1).draw_mermaid_png()))
# analyst_graph

### **Test the Graph**

In [31]:
# Input
max_analysts = 3 
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "1"}}

input_state = { "topic": topic, "max_analysts": max_analysts }

# Run the graph until the first interruption
for event in analyst_graph.stream(input_state, thread, stream_mode="values"):
    # Review
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 100)

Name: Dr. Emily Carter
Affiliation: Tech Innovations Inc.
Role: AI Framework Specialist
Description: Dr. Carter focuses on the technical advantages of adopting LangGraph, emphasizing its modular architecture and ease of integration with existing systems. She is particularly interested in how LangGraph can enhance the efficiency of AI agents in various applications.
----------------------------------------------------------------------------------------------------
Name: Mr. James Liu
Affiliation: Future AI Research Group
Role: AI Ethics Analyst
Description: Mr. Liu examines the ethical implications of using LangGraph as an agent framework. He is concerned with issues such as data privacy, bias in AI decision-making, and the transparency of AI processes, advocating for responsible AI development.
----------------------------------------------------------------------------------------------------
Name: Ms. Sarah Thompson
Affiliation: Business Solutions Consultancy
Role: Business Strategy

In [32]:
# Get state and look at next node
current_state = analyst_graph.get_state(thread)
current_state.next

('human_feedback',)

In [33]:
# Continue the graph execution to end
for event in analyst_graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

--Node--
human_feedback


In [34]:
# If we are satisfied, then we simply supply no feedback
further_feedback = None
analyst_graph.update_state(
    thread, 
    { "human_analyst_feedback": further_feedback}, as_node="human_feedback"
)

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0e7d1f-9cdf-60ca-8003-b51511c63d67'}}

In [35]:
# Continue the graph execution to end
for event in analyst_graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

In [42]:
final_state = analyst_graph.get_state(thread)
analysts = final_state.values.get("analysts")

In [43]:
final_state.next

()

In [44]:
for analyst in analysts:
    print(f"Name: {analyst.name}")
    print(f"Affiliation: {analyst.affiliation}")
    print(f"Role: {analyst.role}")
    print(f"Description: {analyst.description}")
    print("-" * 50) 

Name: Dr. Emily Carter
Affiliation: Tech Innovations Inc.
Role: AI Framework Specialist
Description: Dr. Carter focuses on the technical advantages of adopting LangGraph, emphasizing its modular architecture and ease of integration with existing systems. She is particularly interested in how LangGraph can enhance the efficiency of AI agents in various applications.
--------------------------------------------------
Name: Mr. James Liu
Affiliation: Future AI Research Group
Role: AI Ethics Analyst
Description: Mr. Liu examines the ethical implications of using LangGraph as an agent framework. He is concerned with issues such as data privacy, bias in AI decision-making, and the transparency of AI processes, advocating for responsible AI development.
--------------------------------------------------
Name: Ms. Sarah Thompson
Affiliation: Business Solutions Consultancy
Role: Business Strategy Consultant
Description: Ms. Thompson analyzes the business benefits of adopting LangGraph, focusing

## **Conduct Interview**

### **Generate Question**

- **The analyst will ask questions to the expert.**

In [91]:
class InterviewState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]    # Messages
    max_num_turns: int                                      # Number turns of conversation
    context: Annotated[list, add_messages]                  # Source docs
    analyst: Analysts                                       # Analyst asking questions
    interview: str                                          # Interview transcript
    section: str                                            # Final key we duplicate in outer state for Send() API

    search_query: Annotated[List, add_messages]

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Search query for retrieval.")

In [65]:
def generate_question(state: InterviewState) -> dict:
    """ Generate question for analyst. """
    
    analyst = state['analyst']
    messages = state['messages']

    # generate question
    system_message = SystemMessage(content=QUESTION_INSTRUCTIONS.format(goals=analyst.persona))
    question = llm.invoke([system_message] + messages)

    # write message to state
    return { "messages": [question] }

In [66]:
# in above node change "analyst.persona -> analyst['persona']" for testing below analyst persona
analyst_persona = {
    "persona":
    f"Name: {analyst.name}, \nRole: {analyst.role}, \nDescription: {analyst.description} \nAffiliation: {analyst.affiliation}"
    }
analyst_persona['persona']

'Name: Ms. Sarah Thompson, \nRole: Business Strategy Consultant, \nDescription: Ms. Thompson analyzes the business benefits of adopting LangGraph, focusing on cost-effectiveness, scalability, and the potential for improved customer engagement. She aims to provide insights on how businesses can leverage LangGraph to gain a competitive edge. \nAffiliation: Business Solutions Consultancy'

In [63]:
question = generate_question({
    "analyst": analyst_persona,
    "messages": [],
})

In [74]:
question['messages'][0].content

'Hello Ms. Thompson, my name is Alex Carter, and I’m an analyst focused on understanding innovative business strategies. I’m excited to speak with you today about LangGraph and its potential benefits for businesses. \n\nTo start, could you share some specific examples of how adopting LangGraph has led to cost savings for companies? What are some surprising ways that businesses have found to reduce expenses through its implementation?'

## **Generate Answer: Parallelization**
The expert will gather information from multiple sources in parallel to answer questions.

For example, we can use:

- Specific web sites e.g., via WebBaseLoader
- Indexed documents e.g., via [RAG](https://docs.langchain.com/oss/python/langchain/retrieval)
- Web search
- Wikipedia search

You can try different web search tools, like [Tavily](https://www.tavily.com/).

In [79]:
tavily_search = TavilySearch(max_results=3)

Now, we create nodes to search the web and wikipedia.

We'll also create a node to answer analyst questions.

Finally, we'll create nodes to save the full interview and to write a summary ("section") of the interview.

In [92]:
def web_search(state: InterviewState) -> dict:
    """ Retrieve docs from web search """
    
    message = state['messages']
    
    # search query
    structured_llm = llm.with_structured_output(SearchQuery)
    search_query = structured_llm.invoke(
        [SEARCH_INSTRUCTIONS] + message
    )

    # search
    data = tavily_search.invoke({"query": search_query.search_query})
    search_docs = data.get("results", data)

    # Format
    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in search_docs
        ]
    )

    # Write to state
    return { "context": [formatted_search_docs], "search_query": [search_query] }

In [93]:
web_search_result = web_search({"messages": question["messages"]})

In [98]:
Markdown(web_search_result['search_query'][0].search_query)

cost savings examples LangGraph implementation businesses surprising ways reduce expenses

In [89]:
Markdown(web_search_result['context'][0])

<Document href="https://www.youtube.com/watch?v=9RFNOYtkwsc"/>
How Prosper Cut QA Costs by 90% for Financial Services with LangGraph Agents
LangChain
164000 subscribers
79 likes
4411 views
1 Jul 2025
Learn how Prosper Marketplace transformed their customer call QA process for financial services using LangGraph, reducing verification costs from tens of dollars to cents per call. In this video, see how their AI team built a flexible agent platform that now verifies 100% of customer calls automatically, replacing expensive manual processes.

Key highlights:

- 90%+ cost reduction in QA verification
- From sample-based to 100% call verification automatically
- LangGraph checkpoint system for rapid iteration
- Human-in-the-loop workflows with interrupts
- Scalable multi-agent architecture

🔗 Discover more agent engineer stories: langchain.com/customers
4 comments

</Document>

---

<Document href="https://www.metacto.com/blogs/langgraph-pricing-explained-a-deep-dive-into-integration-maintenance-costs"/>
This guide offers a detailed look at the financial and technical investments required for leveraging LangGraph in your AI-powered
</Document>

---

<Document href="https://rasa.com/blog/cutting-ai-assistant-costs-the-power-of-enhancing-llms-with-business"/>
While studying different approaches, we found that semi-structured approaches separating conversational ability from business logic execution strike the necessary balance, providing reliable and consistent results without sacrificing flexibility. For instance, across our experiments, the LangGraph assistant incurs a mean cost of $0.10 per user message, more than 2 times the CALM Assistant’s mean cost of $0.04. the LangGraph approach lacked consistency and reliability in the conversations, interactions, and execution of required tasks (take a look at the screenshots in this post to get a sense of these mistakes). All conversations with the CALM assistant will move through the business logic as above and will differ only in the type of chit-chat and rephrasing necessary to make the conversation more natural. To verify this across all conversations, we have built end-to-end tests that make sure the business logic is followed and the CALM Assistant passes all such tests. 1. Approaches that separate conversational ability from the execution of business logic, such as CALM, have a significant edge in terms of response time and operational costs.
</Document>

In [99]:
def search_wikipedia(state: InterviewState) -> dict:
    """ Retrieve docs from web search """
    
    message = state['messages']
    
    # search query
    structured_llm = llm.with_structured_output(SearchQuery)
    search_query = structured_llm.invoke(
        [SEARCH_INSTRUCTIONS] + message
    )

    # search
    search_docs = WikipediaLoader(query=search_query.search_query, load_max_docs=2).load()

    # Format
    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata["source"]}" page="{doc.metadata.get("page", "")}"/>\n{doc.page_content}\n</Document>'
            for doc in search_docs
        ]
    )

    # Write to state
    return { "context": [formatted_search_docs], "search_query": [search_query] }

In [100]:
search_wikipedia_result = search_wikipedia({
    "messages": question["messages"]
})